# Habermas Machine data and preprocessing
This follows the instructions in the [Habermas Machine github repo](https://github.com/google-deepmind/habermas_machine), modified to save the processed questions and opinions.

The input to this notebook is the Habermas Machine work, and the output is a dataframe with the questions and opinions `habermas_machine_questions.csv`.

# Setup and Importing Packages


In [ ]:
# Clone github repo locally.
!git clone https://github.com/google-deepmind/habermas_machine

# Adjust path.
import sys
sys.path.insert(0,'/content/habermas_machine')

In [ ]:
# Imports
import ast
import io
import requests

import pandas as pd

from IPython.display import clear_output

# Local imports
from habermas_machine.analysis import live_loading, serialise, types

# Load helper keys used with dataframes.
DFKeys = serialise.SerialisedComparisonKeys
DFGroupedKeys = serialise.GroupedSerialisedComparisonKeys()

In [ ]:
#@title Load all comparison data from Google Cloud Storage.
comparison_data_location = (
    'https://storage.googleapis.com/habermas_machine/datasets/hm_all_candidate_comparisons.parquet'
)
response = requests.get(comparison_data_location)
with io.BytesIO(response.content) as f:
  df_all = pd.read_parquet(f)
clear_output()

df_all.shape # Shape of full comparison data frame.

In [ ]:
#@title Explore data

print("Number of participant sessions (before pre-processing):",
      df_all[DFKeys.COMPARISON_PARTICIPANT_ID].nunique())

print(
    "Number of participant sessions (before pre-processing) of each collection:",
    df_all[
        [DFKeys.COMPARISON_VERSION, DFKeys.COMPARISON_PARTICIPANT_ID]
    ].drop_duplicates().groupby(DFKeys.COMPARISON_VERSION).count()
)

# Example preprocessing

In [ ]:
#@title Select dataset

dataset_name = 'training' # @param ["training", "cohort1_ablation_iid_v1", "cohort2_ablation_iid_v2", "cohort3_ablation_ood_v1", 'cohort4_critique_exclusion', 'cohort5_opinion_exposure', 'cohort6_human_mediator', 'virtual_citizens_assembly']

# Set dataset and parameters based on dataset_name.
if dataset_name == 'training':
  df = df_all[
      df_all[DFKeys.COMPARISON_VERSION].isin([
          'TRAINING_DATA_V1',
          'TRAINING_DATA_V2',
          'TRAINING_DATA_V3',
          'TRAINING_DATA_V4',
          'TRAINING_DATA_V5',
      ])
  ]
  # Backwards incompatibility issue with training data.
  df = df.drop(columns=[
      DFKeys.CANDIDATES_ALL_REWARD_DATA_WELFARE_OR_RANK,
      DFKeys.CANDIDATES_REWARD_DATA_WELFARE_OR_RANK,
  ])
  min_size_parameters = None
  remove_groups_with_repeat_participants = False
  valid_candidate_provenances = (types.ResponseProvenance.MODEL_MEDIATOR,)
elif dataset_name == 'cohort1_ablation_iid_v1':
  df = df_all[df_all[DFKeys.COMPARISON_VERSION] == 'EVAL_COHORT1_ABLATION_IID_V1']
  min_size_parameters = live_loading.GroupMinSizeParameters.ITERATION_EVAL_ABLATION_IID_V1
  remove_groups_with_repeat_participants = True
  valid_candidate_provenances = (types.ResponseProvenance.MODEL_MEDIATOR,)
elif dataset_name == 'cohort2_ablation_iid_v2':
  df = df_all[df_all[DFKeys.COMPARISON_VERSION] == 'EVAL_COHORT2_ABLATION_IID_V2']
  min_size_parameters = live_loading.GroupMinSizeParameters.ITERATION_EVAL_ABLATION_IID_V2
  remove_groups_with_repeat_participants = True
  valid_candidate_provenances = (types.ResponseProvenance.MODEL_MEDIATOR,)
elif dataset_name == 'cohort3_ablation_ood_v1':
  df = df_all[df_all[DFKeys.COMPARISON_VERSION] == 'EVAL_COHORT3_ABLATION_OOD_V1']
  min_size_parameters = live_loading.GroupMinSizeParameters.ITERATION_EVAL_ABLATION_OOD_V1
  remove_groups_with_repeat_participants = True
  valid_candidate_provenances = (types.ResponseProvenance.MODEL_MEDIATOR,)
elif dataset_name == 'cohort4_critique_exclusion':
  df = df_all[df_all[DFKeys.COMPARISON_VERSION] == 'EVAL_COHORT4_CRITIQUE_EXCLUSION']
  min_size_parameters = live_loading.GroupMinSizeParameters.ITERATION_EVAL_CRITIQUE_EXCLUSION
  remove_groups_with_repeat_participants = True
  valid_candidate_provenances = (types.ResponseProvenance.MODEL_MEDIATOR,)
elif dataset_name == 'cohort5_opinion_exposure':
  df = df_all[df_all[DFKeys.COMPARISON_VERSION] == 'EVAL_COHORT5_OPINION_EXPOSURE']
  min_size_parameters = live_loading.GroupMinSizeParameters.ITERATION_EVAL_OPINION_EXPOSURE
  remove_groups_with_repeat_participants = False
  valid_candidate_provenances = (
      types.ResponseProvenance.HUMAN_CITIZEN, # Candidates are other opinions.
  )
elif dataset_name == 'cohort6_human_mediator':
  df = df_all[df_all[DFKeys.COMPARISON_VERSION] == 'EVAL_COHORT6_HUMAN_MEDIATOR']
  min_size_parameters = live_loading.GroupMinSizeParameters.ITERATION_EVAL_HUMAN_MEDIATOR
  remove_groups_with_repeat_participants = False
  # Candidates can be either model or human statements.
  valid_candidate_provenances = (
        types.ResponseProvenance.MODEL_MEDIATOR,
        types.ResponseProvenance.HUMAN_MEDIATOR, )
elif dataset_name == 'virtual_citizens_assembly':
  df = df_all[df_all[DFKeys.COMPARISON_VERSION].isin([
      'EVAL_VIRTUAL_CITIZENS_ASSEMBLY_WEEK3',
      'EVAL_VIRTUAL_CITIZENS_ASSEMBLY_WEEK4',
      'EVAL_VIRTUAL_CITIZENS_ASSEMBLY_WEEK5'
  ])]
  min_size_parameters = None
  remove_groups_with_repeat_participants = False
  valid_candidate_provenances = (types.ResponseProvenance.MODEL_MEDIATOR,)

In [ ]:
#@title Example pre-processing
print('processing', dataset_name)
print('original df shape', df.shape)
live_loading.check_consistent_tuple_lengths_in_grouped_columns(
    df, groups_columns=[
        DFGroupedKeys.OTHER_OPINIONS, DFGroupedKeys.CANDIDATES])

# First, unnest columns (e.g., ratings of statements).
df_unnested = live_loading.unnest_nested_columns(df)
print('unnested df shape', df_unnested.shape)

# Remove rows where OWN_OPINION is not HUMAN_CITIZEN (e.g., MOCKs).
df_unnested = live_loading.filter_on_response_provenances(
    df_unnested,
    provenance_column=DFKeys.OWN_OPINION_PROVENANCE,
    valid_provenances=(types.ResponseProvenance.HUMAN_CITIZEN,),
)
print('filtered df shape after removing invalid opinions', df_unnested.shape)

# Remove rows where CANDIDATES_PROVEANACE is not as expected:
# MODEL_MEDIATOR for most data sets. Can also be HUMAN_CITIZEN or HUMAN_MEDIATOR
# for opinion exposure and human mediator comparison, respectively.
df_unnested = live_loading.filter_on_response_provenances(
    df_unnested,
    provenance_column=DFKeys.CANDIDATES_PROVENANCE,
    valid_provenances=valid_candidate_provenances,
)
print('filtered df shape after removing invalid candidates', df_unnested.shape)

# Remove mock ratings.
df_unnested = live_loading.filter_out_mock_ratings(
    df_unnested, rating_type=live_loading.RatingTypes.AGREEMENT)
print('filtered df shape after removing mock ratings', df_unnested.shape)

# Remove mock rankings.
df_unnested = live_loading.filter_out_mock_rankings(df_unnested)
print('filtered df shape after removing mock rankings', df_unnested.shape)

# Add a column with the numerical equivalents for the Likerts.
df_unnested = live_loading.add_numerical_ratings(df_unnested)
print('added numerical ratings df shape', df_unnested.shape)

# Optional (not used in training or human mediator eval):
# Remove groups with repeat participants.
if remove_groups_with_repeat_participants:
  df_unnested = live_loading.filter_groups_with_repeat_participants(
      df_unnested, 'worker_id')
  print('filtered df after removing groups with repeat participants', df_unnested.shape)

# Renest previously unnested columns.
df_nested = live_loading.nest_columns_as_tuples(df_unnested)
print('renested df shape', df_nested.shape)

# Human Mediator specific preprocessing: Only keep rounds where both human and
# model generated statements.
if dataset_name == 'cohort6_human_mediator':
  df_nested = df_nested[
      df_nested[DFKeys.CANDIDATES_PROVENANCE].apply(len) == 2
  ]
  print('only keeping rounds where both human and model made statement',
        df_nested.shape)

# Optional: Filter by number of groups of min size (pre-registration criteria).
# Note, this should be applied to only a single evaluation dataset and not
# multiple datasets at the same time.
if min_size_parameters is not None:
  df_nested = live_loading.filter_by_number_of_groups_of_min_size(
      df_nested,
      **min_size_parameters.value)
  print('filtered df after setting number of groups of min size', df_nested.shape)

print('Number of groups in preprocessed dataframe:',
      df_nested[DFKeys.LAUNCH_ID].nunique())

In [ ]:
# This will be helpful for debugging.
pd.set_option('display.max_columns', 400)  # Show all columns
pd.set_option('display.max_rows', 400)     # Show all rows

# This is an example of what the data looks like.
df_nested.iloc[0]

In [ ]:
df_questions = df_nested[df_nested['iteration_index'] == 0] # This stops us having duplicates from multiple rounds.

df_questions = df_questions[['question.text','question.topic','own_opinion.text']]

In [ ]:
# Sanity check
df_questions[df_questions['question.text'] == 'Are human beings the most intelligent life form in the Universe?']


In [ ]:
grouped_question_to_opinions = df_questions.groupby('question.text')['own_opinion.text']
    
# Let's check that each question has no duplicates.
assert (grouped_question_to_opinions.apply(lambda x: x.count() / x.drop_duplicates().count()) == 1).all()

assert (df_questions.groupby('question.text')['question.topic'].nunique() == 1).all()

In [ ]:
df_opinions_wrapped = df_questions.groupby('question.text')['own_opinion.text'].apply(lambda x: x.to_list()).to_frame().reset_index()

question_to_topic = df_questions.set_index('question.text')['question.topic'].to_dict()

df_opinions_wrapped['question_topic'] = df_opinions_wrapped['question.text'].map(question_to_topic)

In [ ]:
# Let's add some unique question ids to make life easier later.
df_opinions_wrapped['question_id'] = df_opinions_wrapped.groupby('question.text').ngroup()

In [ ]:
# Save to csv
df_opinions_wrapped.to_csv('data/habermas_machine_questions.csv', index=False)